##### 01 数据集导入，创建训练集和测试集。通过查看注释，理解整个流程。

In [1]:
import glob
import numpy as np
# 训练集、测试集划分
from sklearn.model_selection import train_test_split
# 数据集相对路径
DATA_PATH = "../../../10_tingml_datasets/"
# LABELS 的内容尽量与前面store_data.py保持一致
LABELS = ["Stationary", "Tilted", "Rotating", "Moving"]
# 代表一个样本内容，如连续10次传感器读到的6轴数据作为一个样本
SAMPLES_PER_GESTURE = 10
def load_one_label_data(label):
    path = DATA_PATH + label + "*.npy"
    files = glob.glob(path)
    datas = []
    for file in files:
        try:
            data = np.load(file)
            # 切除多余数据，如数据当中有61份，但每个样本只需要10份，那么最后一份需要丢弃。
            num_slice = len(data) // SAMPLES_PER_GESTURE
            datas.append(data[: num_slice * SAMPLES_PER_GESTURE, :])
        except Exception as e:
            print(e)
    datas = np.concatenate(datas, axis=0)
    # 由于本案例给的是全连接层，输入为1维数据。(其余如conv需要自行根据模型输入修改尺寸，如二维)
    # MLP
    # datas = np.reshape(datas,(-1, 6 * SAMPLES_PER_GESTURE,),)  # Modified here
    # CNN 1
    datas = np.reshape(datas,(-1, 6 * SAMPLES_PER_GESTURE, 1),)  # Modified here
    idx = LABELS.index(label)
    labels = np.ones(datas.shape[0]) * idx
    return datas, labels
all_datas = []
all_labels = []
# 导入每个label对应的数据
for label in LABELS:
    datas, labels = load_one_label_data(label)
    all_datas.append(datas)
    all_labels.append(labels)
dataX = np.concatenate(all_datas, axis=0)
dataY = np.concatenate(all_labels, axis=0)
# 输入和样本到此创建完毕

# 训练集、测试集划分
# test_size 表示数据集里面有20%将划分给测试集
# stratify=dataY指定按label进行划分, 确保数据集划分公平
xTrain, xTest, yTrain, yTest = train_test_split(
    dataX, dataY, test_size=0.2, stratify=dataY
)
print(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)

(1312, 60, 1) (328, 60, 1) (1312,) (328,)


##### 02 模型创建
下面将创建很简单的多层感知机模型，后续可自行定义模型结构。需要根据自身需求，自行上网查询其他模型，如CNN，切记模型不要太大，嵌入式设备大致提供32K空间供运行模型。
模型需要注意输入尺寸，如CNN往往多维数据，如**Conv1d 输入二维，可将输入改为(6 * SAMPLES_PER_GESTURE,1)或者(SAMPLES_PER_GESTURE, 6), 上面数据集对应尺寸也需要修改**

In [2]:
import os
# 0 = INFO, 1 = WARNING, 2 = ERROR, 3 = FATAL
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import tensorflow.keras as keras

# 设置环境变量，控制日志级别
def mlp():
    # 一个用于线性堆叠多个网络层的模型。
    # Sequential模型是最简单的神经网络模型，它按照层的顺序依次堆叠，每一层的输出会成为下一层的输入。
    model = keras.Sequential()
    # 第一层, 添加全连接层，输出尺寸为64，激活函数采用"relu"
    # 第一层需要制定输入大小，这里和数据集对应input_shape=(6 * SAMPLES_PER_GESTURE,)
    model.add(keras.layers.Dense(64, activation="relu", input_shape=(6 * SAMPLES_PER_GESTURE,)))
    # 添加池化层，防止模型过拟合，每次自动忘记20%的参数
    model.add(keras.layers.Dropout(0.2))
    # 最后一层，全连接层，输出尺寸对应labels数量，激活函数采用"softmax"
    # softmaxs输出的结果代表每个label的概率，如第0个代表label 0的概率
    model.add(keras.layers.Dense(len(LABELS), activation="softmax"))
    return model
def cnn():
    # 一个用于线性堆叠多个网络层的模型。
    # Sequential模型是最简单的神经网络模型，它按照层的顺序依次堆叠，每一层的输出会成为下一层的输入。
    model = keras.Sequential()
    # 注意CNN与MLP的输入shape
    # 16个输出通道，3为卷积核大小
    model.add(
        keras.layers.Conv1D(
            8,3,padding="same",activation="relu",input_shape=(6 * SAMPLES_PER_GESTURE, 1),
        )
    )
    model.add(keras.layers.Conv1D(8, 3, padding="same", activation="relu"))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(8, activation="relu"))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(LABELS), activation="softmax"))
    return model


model = cnn()
# 打印模型结构
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60, 8)             32        
                                                                 
 conv1d_1 (Conv1D)           (None, 60, 8)             200       
                                                                 
 global_average_pooling1d (  (None, 8)                 0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                        

##### 03 模型训练及测试

In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint
# 加载模型
from tensorflow.keras.models import load_model
# 测试模型性能
from sklearn.metrics import confusion_matrix
# 模型训练优化器，学习率为0.001
optimizer = keras.optimizers.Adam(lr=0.001)
# 制定模型优化器，和损失函数、评价指标
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["sparse_categorical_accuracy"],
)
# 制定保存模型的路径
filepath = "best_model.h5"
# 训练时，保存最好模型
checkpoint = ModelCheckpoint(
    filepath,
    monitor="val_sparse_categorical_accuracy",
    verbose=1,
    save_best_only=True,
    mode="max",
)
# 模型训练, 训练集，batch_size为批大小，可提高训练速度
# validation_data指明验证集，epochs表示训练迭代轮数
# verbose=1表示打印训练日志
# callbacks调用上述保存模型的方法
history = model.fit(
    xTrain,
    yTrain,
    batch_size=8,
    validation_data=(xTest, yTest),
    epochs=50,
    verbose=1,
    callbacks=[checkpoint],
)
# 至此模型训练完毕

Epoch 1/50
127/164 [======================>.......] - ETA: 0s - loss: 1.4286 - sparse_categorical_accuracy: 0.3671
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.57622, saving model to best_model.h5
164/164 [==============================] - 1s 2ms/step - loss: 1.3291 - sparse_categorical_accuracy: 0.4123 - val_loss: 0.9200 - val_sparse_categorical_accuracy: 0.5762
Epoch 2/50
136/164 [=======================>......] - ETA: 0s - loss: 0.9461 - sparse_categorical_accuracy: 0.5570

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_sparse_categorical_accuracy improved from 0.57622 to 0.73171, saving model to best_model.h5
164/164 [==============================] - 0s 1ms/step - loss: 0.9351 - sparse_categorical_accuracy: 0.5877 - val_loss: 0.8473 - val_sparse_categorical_accuracy: 0.7317
Epoch 3/50
130/164 [======================>.......] - ETA: 0s - loss: 0.8749 - sparse_categorical_accuracy: 0.7087
Epoch 3: val_sparse_categorical_accuracy improved from 0.73171 to 0.77744, saving model to best_model.h5
164/164 [==============================] - 0s 1ms/step - loss: 0.8593 - sparse_categorical_accuracy: 0.7149 - val_loss: 0.7732 - val_sparse_categorical_accuracy: 0.7774
Epoch 4/50
138/164 [========================>.....] - ETA: 0s - loss: 0.7811 - sparse_categorical_accuracy: 0.7337
Epoch 4: val_sparse_categorical_accuracy improved from 0.77744 to 0.78354, saving model to best_model.h5
164/164 [==============================] - 0s 1ms/step - loss: 0.7860 - sparse_categorical_accuracy: 0.7195 - val_lo

In [4]:
# 加载模型
model = load_model(filepath)
# 模型推理，预测
predictions = model.predict(xTest)
predictions = np.argmax(predictions, axis=1)
# 查看混淆矩阵，效果越好，预测则集中在对角线。
cm = confusion_matrix(yTest, predictions)
print(cm)

11/11 [==============================] - 0s 1ms/step
[[ 60   0   0   0]
 [  0  67   0   1]
 [  0   0 105   3]
 [  0   0   1  91]]


由于运动状态简单，上述最简单的模型可能也会获得不错的性能，当运动状态变得复杂，上述MLP模型性能将很难满足需求。
#### 04 生成最终部署的模型
由于RIOT系统使用的时tflite-micro库且资源有限，将模型量化，并保存成tflite-micro可识别的格式，注意
`data_test = np.reshape(data_test, (-1, 6 * SAMPLES_PER_GESTURE, ))`后的尺寸维度和大小需要和前面大致对应

In [5]:
# Convert the model to the TensorFlow Lite format with quantization
# 加载模型
model = load_model(filepath)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# 保存初始版本，后续对比用
open("model_basic.tflite", "wb").write(tflite_model)

# 量化模型, 定义输入格式与大小，只需要修改(-1, 6 * SAMPLES_PER_GESTURE,)与上面对应即可，其余不用变
data_test = xTest.astype("float32")
# np.reshape 和一开始数据集导入对应
# MLP
# data_test = np.reshape(data_test, (-1, 6 * SAMPLES_PER_GESTURE, ))
# CNN 1
data_test = np.reshape(data_test, (-1, 6 * SAMPLES_PER_GESTURE, 1))
data_ds = tf.data.Dataset.from_tensor_slices((data_test)).batch(1)
# Rest of your code...
def representative_data_gen():
    for input_value in data_ds.take(100):
        yield [input_value]
converter.representative_dataset = representative_data_gen
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

# 量化前后对比
basic_model_size = os.path.getsize("model_basic.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("model.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

INFO:tensorflow:Assets written to: /tmp/tmp2fwcvshz/assets


INFO:tensorflow:Assets written to: /tmp/tmp2fwcvshz/assets


INFO:tensorflow:Assets written to: /tmp/tmpc53vlkob/assets


INFO:tensorflow:Assets written to: /tmp/tmpc53vlkob/assets


Basic model is 5576 bytes
Quantized model is 5984 bytes
Difference is -408 bytes


/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


前面，通过量化，帮助我们模型节省了2440Bytes大小。需要验证量化后的模型输入格式和尺寸是否正确

In [6]:
# Now let's verify the model on a few input digits
# Instantiate an interpreter for the model
model_quantized_reloaded = tf.lite.Interpreter("model.tflite")

# Allocate memory for each model
model_quantized_reloaded.allocate_tensors()

# Get the input and output tensors so we can feed in values and get the results
model_quantized_input = model_quantized_reloaded.get_input_details()[0]["index"]
model_quantized_output = model_quantized_reloaded.get_output_details()[0]["index"]
# Create arrays to store the results
model_quantized_predictions = np.empty(xTest.size)
for i in range(20):
    # Reshape the data and ensure the type is float32
    # test_data = np.reshape(
    #     xTest[i],
    #     (
    #         1,
    #         6 * SAMPLES_PER_GESTURE,
    #         1,
    #     ),
    # ).astype("float32")
    test_data = np.expand_dims(xTest[i], axis=0).astype("float32")
    print(test_data.shape)
    # Invoke the interpreter
    model_quantized_reloaded.set_tensor(model_quantized_input, test_data)
    model_quantized_reloaded.invoke()
    model_quantized_prediction = model_quantized_reloaded.get_tensor(
        model_quantized_output
    )

    print("Digit: {} - Prediction:\n{}".format(yTest[i], model_quantized_prediction))
    print("")

(1, 60, 1)
Digit: 2.0 - Prediction:
[[0.         0.         0.99609375 0.        ]]

(1, 60, 1)
Digit: 0.0 - Prediction:
[[0.9375     0.05859375 0.00390625 0.        ]]

(1, 60, 1)
Digit: 3.0 - Prediction:
[[0.         0.00390625 0.         0.99609375]]

(1, 60, 1)
Digit: 2.0 - Prediction:
[[0.         0.         0.99609375 0.        ]]

(1, 60, 1)
Digit: 0.0 - Prediction:
[[0.9375     0.05859375 0.00390625 0.        ]]

(1, 60, 1)
Digit: 3.0 - Prediction:
[[0.         0.         0.00390625 0.99609375]]

(1, 60, 1)
Digit: 3.0 - Prediction:
[[0.         0.         0.05859375 0.94140625]]

(1, 60, 1)
Digit: 3.0 - Prediction:
[[0.         0.00390625 0.         0.99609375]]

(1, 60, 1)
Digit: 2.0 - Prediction:
[[0.         0.         0.99609375 0.        ]]

(1, 60, 1)
Digit: 0.0 - Prediction:
[[0.9375     0.05859375 0.00390625 0.        ]]

(1, 60, 1)
Digit: 2.0 - Prediction:
[[0.         0.         0.99609375 0.00390625]]

(1, 60, 1)
Digit: 3.0 - Prediction:
[[0.00390625 0.93359375 0.003

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
